In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

# Create a dash application
app = dash.Dash(__name__)

# Build dash app layout
app.layout = html.Div(children=[ html.H1('Flight Delay Time Statistics', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 30}),
                                html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', 
                                type='number', style={'height':'35px', 'font-size': 30}),], 
                                style={'font-size': 30}),
                                html.Br(),
                                html.Br(), 
                                # Segment 1
                                html.Div([
                                        html.Div(dcc.Graph(id='carrier-plot')),
                                        html.Div(dcc.Graph(id='weather-plot'))
                                ], style={'display': 'flex'}),
                                # Segment 2
                                html.Div([
                                        html.Div(dcc.Graph(id='nas-plot')),
                                        html.Div(dcc.Graph(id='security-plot'))
                                ], style={'display': 'flex'}),
                                # Segment 3
                                html.Div(dcc.Graph(id='late-plot'), style={'width':'65%'})
                                ])

""" Compute_info function description

This function takes in airline data and selected year as an input and performs computation for creating charts and plots.

Arguments:
    airline_data: Input airline data.
    entered_year: Input year for which computation needs to be performed.
    
Returns:
    Computed average dataframes for carrier delay, weather delay, NAS delay, security delay, and late aircraft delay.

"""
def compute_info(airline_data, entered_year):
    # Select data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late

"""Callback Function

Function that returns fugures using the provided input year.

Arguments:

    entered_year: Input year provided by the user.
    
Returns:

    List of figures computed using the provided helper function `compute_info`.
"""
# Callback decorator
@app.callback( [
               Output(component_id='carrier-plot', component_property='figure'),
               Output(component_id='weather-plot', component_property='figure'),
               Output(component_id='nas-plot', component_property='figure'),
               Output(component_id='security-plot', component_property='figure'),
               Output(component_id='late-plot', component_property='figure')
               ],
               Input(component_id='input-year', component_property='value'))
# Computation to callback function and return graph
def get_graph(entered_year):
    
    # Compute required information for creating graph from the data
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_info(airline_data, entered_year)
            
    # Line plot for carrier delay
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
    # Line plot for weather delay
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
    # Line plot for nas delay
    nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
    # Line plot for security delay
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
    # Line plot for late aircraft delay
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
            
    return[carrier_fig, weather_fig, nas_fig, sec_fig, late_fig]

# Run the app
if __name__ == '__main__':
    app.run_server()

In [7]:
data.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [10]:
sites = data['Launch Site'].unique()
sites

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [49]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the data using pandas
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
app.title = "SpaceX Launch Dashboard"

# List of launch sites
launch_sites = data['Launch Site'].unique()

# Create color map for each launch site
color_map = {
    'CCAFS LC-40': '#1f77b4',
    'VAFB SLC-4E': '#ff7f0e',
    'KSC LC-39A': '#2ca02c',
    'CCAFS SLC-40': '#d62728'
}

# Create the layout of the app
app.layout = html.Div([
    html.H1("SpaceX Launch Dashboard", style={'textAlign': 'left', 'color': '#503D36', 'font-size': 24}),
    html.Div([
        html.Label("Select a Launch Site:"),
        dcc.Dropdown(
            id='site-dropdown',
            options=[
                {'label': 'All Sites', 'value': 'ALL'}
            ] + [{'label': site, 'value': site} for site in launch_sites],
            value='ALL',
            placeholder="Select a Launch Site here",
            searchable=True
        )
    ]),
    html.Div(id='output-container', className='chart-grid', style={'display': 'flex', 'flex-wrap': 'wrap'}),
    dcc.Graph(id='success-pie-chart'),
    html.Label('Payload Mass Range (kg)'),
    dcc.RangeSlider(
        id='range-slider',
        min=data['Payload Mass (kg)'].min(),
        max=data['Payload Mass (kg)'].max(),
        step=1000,
        marks={i: str(i) for i in range(int(data['Payload Mass (kg)'].min()), int(data['Payload Mass (kg)'].max()) + 1001, 1000)},
        value=[int(data['Payload Mass (kg)'].min()), int(data['Payload Mass (kg)'].max())]
    ),
    dcc.Graph(id='success-payload-scatter-chart')
])

# Define callback to update the pie chart based on the selected launch site
@app.callback(Output('success-pie-chart', 'figure'),
              [Input('site-dropdown', 'value'),
               Input('range-slider', 'value')])
def update_pie_chart(selected_site, range_value):
    if selected_site == 'ALL':
        filtered_data = data[(data['Payload Mass (kg)'] >= range_value[0]) & 
                             (data['Payload Mass (kg)'] <= range_value[1])]
        fig = px.pie(filtered_data, names='Launch Site', title='Success Rate for All Sites',
                     color='Launch Site', color_discrete_map=color_map,
                     hole=0.3)  # hole creates a donut-like pie chart
    else:
        filtered_data = data[(data['Launch Site'] == selected_site) &
                             (data['Payload Mass (kg)'] >= range_value[0]) & 
                             (data['Payload Mass (kg)'] <= range_value[1])]
        fig = px.pie(filtered_data, names='class', title=f'Success Rate for {selected_site}',
                     color_discrete_map={0: color_map[selected_site], 1: color_map[selected_site]},
                     hole=0.3)  # hole creates a donut-like pie chart
    return fig

# Define callback to update the scatter plot based on the selected launch site and payload range
@app.callback(Output('success-payload-scatter-chart', 'figure'),
              [Input('site-dropdown', 'value'),
               Input('range-slider', 'value')])
def update_scatter_chart(selected_site, payload_range):
    if selected_site == 'ALL':
        filtered_data = data[(data['Payload Mass (kg)'] >= payload_range[0]) & 
                             (data['Payload Mass (kg)'] <= payload_range[1])]
        fig = px.scatter(filtered_data, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Payload Success Rate for All Sites',
                         labels={'Payload Mass (kg)': 'Payload Mass (kg)', 'class': 'Class'},
                         hover_data={'Booster Version Category': True},
                         color_discrete_map=color_map)
    else:
        filtered_data = data[(data['Launch Site'] == selected_site) &
                             (data['Payload Mass (kg)'] >= payload_range[0]) & 
                             (data['Payload Mass (kg)'] <= payload_range[1])]
        fig = px.scatter(filtered_data, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title=f'Payload Success Rate for {selected_site}',
                         labels={'Payload Mass (kg)': 'Payload Mass (kg)', 'class': 'Class'},
                         hover_data={'Booster Version Category': True},
                         color_discrete_map=color_map)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [20]:
data.columns

Index(['Date', 'Year', 'Month', 'Recession', 'Consumer_Confidence',
       'Seasonality_Weight', 'Price', 'Advertising_Expenditure', 'Competition',
       'GDP', 'Growth_Rate', 'unemployment_rate', 'Automobile_Sales',
       'Vehicle_Type', 'City'],
      dtype='object')